# 抽象表: ADT List 及其实现

可以认为 ADT 是一个抽象的数学概念. 我们定义它为一个由基本元素构成的集合,
以及相应的操作(operation). 比如我们可以定义一个数据类型名为 set,
一个 set 可以包含有限个或零个元素. 我们可以定义操作为 add, 即增加一个指定的元素；
操作 remove, 删除一个指定的元素；操作 size, 计算当前元素总数, 等等.

不同的编程语言, 都不难将上述概念具体化. 比如将元素具体为一个浮点数,
然后我们可以讨论如何实现一个有限的浮点数集, 并将其用于我们具体的问题中. 而 C++
提供了面向对象设计(object-oriented)的能力, 因此可以将这件事情做的更加直接. 一个
C++ 类(class), 允许将数据结构和操作捆绑在一起定义, 因此几乎可以直接视为一个 ADT
的实现.

## 抽象数据结构: 表(List)

我们称有限个元素组成的序列 $A_0$, $A_1$, $\cdots$, $A_{N - 1}$ 为一个 ADT List.
表的长度 (size) 为非负整数 $N$. 如果 $N == 0$ 为真, 我们称该表为空表(empty list).

对非空表中元素 $A_i$, $0 \leq i < N$, $A_{i - 1}$ 称为它的前任 (predecessor),
$A_{i + 1}$ 称为它的继任 (successor). 当然, 前任和继任可以不存在. 而自然数 $i$ 称为
$A_i$ 的位置 (position).


我们可以定义以下一些常见的操作:
+ `printList` 输出表中全部元素. 空表则什么也不输出.
+ `makeEmpty` 清空列表, 删除全部元素. 空表则什么也不发生. 
+ `find (x)` 这里 `x` 是一个具体的值, 表中寻找值 `x`, 如果找到, 返回第一个找到的 `x` 的位置, 否则返回 NIL. 
+ `insert (x, pos)` 这里 `x` 是一个具体的值, `pos` 是一个位置, 即在指定位置 `pos` 前插入一个新元素 `x`, 插入之后新元素 `x` 在表中的位置将会是 `pos`. 如果 `pos` 位置不存在, 则报错.
+ `remove (x)` 删除 `x`, 如果表中没有 `x` 则什么事也不发生. 这里我们可以规定, 如果表中不止一个 `x`, 一次只删除第一个发现的那一个. 也即这里其实是先操作了一次 `find(x)`. 
+ `findKth (pos)` 返回第 `pos` 位置的元素. 如果位置 `pos` 不存在, 则报错.
+ `next (pos)` 返回第 `pos` 位置的继任元素的位置. 如果继任不存在, 则返回 NIL.
+ `previous (pos)` 返回第 `pos` 位置的前任元素的位置. 如果前任不存在, 则返回 NIL.

我们还可以根据需要, 定义和扩展表的操作功能. 而对于不同的具体实现, 这些功能定义也可能得不到严格的遵守(因此需要作出调整和改变).

在以上操作中, 我们发现有些操作是不会改变表的内容和结构的, 如`printList` 和 `find(x)`, 这些操作我们一般称为**静态操作**; 而有些操作则会改变, 如 `insert` 和 `remove`. 这里我们称其为**动态操作**.

## 表的简单数组(Array)实现

**数组(Array)** 是指用一段**连续**的内存空间来存放一个表. 由于表的每一个元素所占内存空间相等, 因此我们只需记录表的第一个元素所在内存地址, 那么就可以容易地获得数组的第 $k$ 个元素的内存地址. 在 C / C++ 中, 这个 `findKth` 的操作, 直接通过运算 `[]` 实现. 这种实现不但非常简单, 而且是计算机存储在系统层面上最基础的方式, 本质上, 我们的全部内存都可以看作按内存地址连续存放的一个大数组.

得益于数据在内存中的连续存放, 静态操作如 `findKth` 的时间代价一般为 $O(1)$, 而动态操作如 `insert` 和 `remove` 的代价是 $O(N)$. 这是因为我们在改变了表的结构之后, 必须确保数据仍然有序和连续存放. 

换言之, 用 Array 去实现 ADT List, 静态效率较高而动态效率较低. 

我们接下去用 C++ 实现这个操作, 主要目的是用这个简单的模型, 学习 C++ 的基本框架.

我们先考虑一个只能存放 `char` 类型的数组, 数组实现的关键是必须保持数据在内存中的连续性, 然后利用这一点加速静态操作. 但抛开这些具体的实现细节不管, 我们先实现我们的 List 的抽象概念定义: 

In [1]:
class ArrayList
{
private:
public:
    void printList();
    void makeEmpty();
    int find(char x);
    void insert(char x, int pos);
    void remove(char x);
    char findKth(int pos);
    int next(int pos);
    int previous(int pos);
};

写完这个定义我们就知道, `next` 和 `previous` 这两个操作是愚蠢的. 这里本质上我们唯一需要做的事, 是去判断一下一个 `pos` 是否越界, 是否合法. 所以我们先去掉这两个操作, 如果有必要, 增加一个 `pos` 合法性判断的操作. 这种概念上的讨论, 越早进行越好, 越到后续会有大量沉没成本.

然后我们这里没有考虑具体将数据存放在什么地方, 由于我们这里从学习角度出发, 放弃 C++ 已经存在的数据结构, 所以干脆直接从向系统申请内存开始. 这样我们需要一个指针来记录数据在内存中开始的地址. 将上述代码调整如下: 

In [2]:
class ArrayList
{
private:
    char* data;
public:
    void printList();
    void makeEmpty();
    int find(char x);
    void insert(char x, int pos);
    void remove(char x);
    char findKth(int pos);
};

在这里我们遗漏了两个必须要有的操作: 出生和死亡. 也即对任何一个类而言, 都需要考虑, 
+ 当一个实例被创建和初始化数据时；
+ 当一个实例被删除时；

我们需要做的事. 在 C++ 的规范中, 任何类都必须有以下两个成员操作:
+ **构造函数, constructor**. 函数名和类名一致, 没有返回类型. 参数可以自定. 这是在创建实例时, 会第一个调用的函数,因此可以在这里规定实例的数据如何初始化.
+ **析构函数, destructor**. 函数名为\~加类名, 同样没有返回类型, 也没有参数. 这是当一个实例正常删除时, 最后被调用的一个函数, 在这里可以做一些收尾工作, 比如确保动态分配的内存都已经安全释放.

如果没有显式声明构造和析构函数, 那么编译器也会默认产生一个缺省的构造函数和析构函数. 它们的都相当于一个带空参数的空函数. 构造函数可以重载, 但析构不会重载(为什么?).  

在 ArrayList 中, 我们需要在构造函数中申请数据内存, 而在析构函数中确保这部分数据被正确释放, 所以默认的构造和析构不一定合适. 我们做以下的设计: 

In [2]:
#include <iostream>

class ArrayList
{
private:
    // C++ 11 以后允许给类成员赋初指, 且发生在任何构造函数之前.
    char* data = nullptr;
public:
    void printList();
    void makeEmpty();
    int find(char x);
    void insert(char x, int pos);
    void remove(char x);
    char findKth(int pos);
    ArrayList() {  }
    ArrayList(int N) { data = new char[N]; };
    ~ArrayList() { if (data != nullptr) delete [] data; }
};

这里我们实际上重载了构造函数, 也就是提供了两个不同版本的构造函数, 它们通过参数调用区别. 这也是析构函数为何不能重载的原因, 它没有参数. 尽管以上代码都是可行的, 但逻辑上这里引入了另一个问题: 我们如何确保数组大小是正确的. 这一点无法在 C++ 内部解决. 因为一旦动态内存分配了, 它就只是一个指针, 我们不能再通过任何语言内的机制去调查它的实际内存占用. 这是因为 C++ 在这一部分基本还是继承了 C 语言的机制, 而 C 语言作为一种直接面向硬件编程的语言, 有时效率比严谨更加重要(这一点现在越来越有疑问). 具体而言, 我们这里要让用户能合理得知数组大小的办法几乎只有增加一个属性, 但同时我们要完全依赖程序员确保这个表示大小的属性值总是对的. 如果程序员出错了, 没有任何客观机制中止这一点. 这就表明, 至少到现在为止, C++ 还不是一种完全客观安全的语言, 需要我们时刻注意类似内存泄漏等可能的隐患. C++ 确实在这些方面越来越注意. 比如通过 **const** 等限定词约束成员函数的行为. 

这里顺便也交待一下命名问题. 基本上只要意义明确即可, 尽量减少无意义的命名. 比如这里的 `pos` 是可以接受的, 而 `x` 和 `N` 则多少有点迷惑. 各函数的命名都用了驼峰风格, 基本上都是动词属性. 这些都供大家参考. 总之, 尽量形成一个你自己可以接受的, 比较合理且封闭的风格, 然后一以贯之就行. 

基于上述描述, 我们继续进化我们的 ArrayList.

In [35]:
#include <iostream>

class ArrayList
{
private:
    // C++ 11 以后允许给类成员赋初指, 且发生在任何构造函数之前.
    char* data = nullptr;
    // 用以表示数组大小, 必须靠程序员可靠维护.
    int size = 0;
public:
    // 这个 const 表示该成员函数不会修改类的任何数据.
    void printList() const;
    void makeEmpty();
    // 从实用角度, find 返回一个指针会更有意义, 所以这里也提供了动静两套版本.
    const char* find(char val) const ;  
    char* find(char val) ;
    // 原本的 find 保留, 但改名为 findIdx.
    int findIdx(char val) const;
    // 动态函数就不可以加 const 限制.
    void insert(char val, int pos);
    void remove(char val);
    // 不允许修改返回的指针, 也不允许函数修改类成员数据. 
    const char* findKth(int pos) const;
    // 这是上一个函数的重载, 允许修改返回的指针, 也允许修改函数成员的数据.
    char* findKth(int pos);
    ArrayList() { }
    ArrayList(int N) { data = new char[N]; };
    ~ArrayList() { if (data != nullptr) delete [] data; }
};

在这个版本中, 我们注意一种新的重载约定. 一般函数重载是通过函数参数决定的, 但在 C++ 中, 当你对一个`const`对象或者一个非`const`对象调用`fun()`函数时，编译器会根据调用对象的`const`情况来决定调用哪个版本的函数。相比上一个版本的 `findKth`, 只读的版本少了一次数据复制, 而可写的版本事实上提供了进一步修改找到的数据的可能性. 是否要提供这种可能性, 是设计者要考虑的问题. 这里可以考虑将处在同一地位的 `find` 也改造成这样的双接口形式. 而将原本的 `find` 改名为 `findIdx`.

对于以上成员函数的实现, 不是一个困难的任务, 这里只介绍一些要点: 
+ 要注意一些异常情况的判定和排除.

In [36]:
void ArrayList::makeEmpty()
{
    if (size == 0)
    {
        // 警告, 这个情况虽然不合逻辑, 但不能完全排除在合理情形外.
        std::cerr << "Warning: The list is already empty." << std::endl;
        return;
    }
    else
    {
        size = 0;
        if (data != nullptr)
        {
            delete [] data;
            data = nullptr;
        }
        else
        {
            // 错误, 这是一个逻辑上的错误, 直接退出.  
            std::cerr << "Error: A non-empty list has a null pointer." << std::endl;
            exit(-1);
        }
    }
    return;
}

+ 不要出现同一功能的重复代码.

In [38]:
const char* ArrayList::find(char val) const  
{  
    for (int i = 0; i < size; ++i) {  
        if (data[i] == val)  
        {  
            return &data[i];  
        }  
    }  
    return nullptr;  
}  
  
char* ArrayList::find(char val)   
{  
   return const_cast<char*>(static_cast<const ArrayList&>(*this).find(val));  
}  

input_line_62:14:1: error: function definition is not allowed here
{  
^


Interpreter Error: 

这里别管在线编译器的错误. 这段代码体现了的原则是相同的代码永远不要重复出现. 这里静态和动态的版本逻辑上除了权限以外, 会完全一致, 但是将其中一个版本完全复制一遍给另一个版本用, 除了会留下数据冗余的隐患, 更麻烦的是在一些多线程等并行场合, 会发生无法预计的错误. 一个基本原则是, 动态版本去复用静态版本. 中间用合适的类型转换. 比如这里, `static_cast<const ArrayList&>(*this)` 先做了第一次转换, 将"自己", `*this` 用 `static_cast` 转换成一个 `const ArrayList&`. 然后我们就可以调用静态版本的 `find`, 得到的结果也是一个 `const char *`, 所以再用 `const_cast<char*>` 去掉它的 `const` 属性. 这样避免了代码重复, 同时, 相对而言, 静态版本更安全. 类似地, `findKth` 也有同样的考虑.

In [39]:
const char* ArrayList::findKth(int pos) const
{
    if (pos < 0 || pos >= size)
    {
        std::cerr << "Error: Invalid position." << std::endl;
        exit(-1);
    }
    return data + pos;
}

char* ArrayList::findKth(int pos) 
{
    return const_cast<char*>(static_cast<const ArrayList&>(*this).findKth(pos));  
}

input_line_63:12:1: error: function definition is not allowed here
{
^


Interpreter Error: 

一个数据结构的实现难点一般都集中在动态操作部分, 也就是 `insert` 和 `remove` 等操作. 数组实现的线性表本质复杂度是 $O(N)$, 这一点无法改变, 但一些细节调整, 比如考虑 `insert` 或 `remove` 有一定概率连续出现, 因此尽量减少重新申请内存的次数等等. 但作为一个 C++ 入门级别的示例, 直观, 展示语法细节的需求高于非本质的效率提升, 因此我们先实现一个非常简单直接的版本, 而将改进放到后面去讨论. 

In [40]:
void ArrayList::insert(char val, int pos)
{
    if (pos < 0 || pos > size)
    {
        std::cerr << "Error: Invalid position." << std::endl;
        exit(-1);
    }
    if (size == 0)
    {
        if (data == nullptr)
            data = new char[1];
        else
        {
            std::cerr << "Warning: Allocating memory to a non-null pointer may result in memory leakage." 
            << std::endl;
            exit(-1);
        }
        data[0] = val;
        size++;
        return;
    }
    char* p = new char[size + 1];
    for (int i = 0; i < pos; i++)
        p[i] = data[i];
    p[pos] = val;
    for (int i = pos + 1; i < size + 1; i++)
        p[i] = data[i - 1];
    delete [] data;
    data = p;
    size++;
    return;
}

void ArrayList::remove(char val)
{
    int pos = findIdx(val);
    if (pos == -1)
    {
        std::cerr << "Error: The value is not in the list." << std::endl;
        exit(-1);
    }
    for (int i = pos + 1; i < size; i++)
        data[i - 1] = data[i];
    // 只改了 size, 没有释放内存.
    size--;
}

input_line_64:35:1: error: function definition is not allowed here
{
^


Interpreter Error: 

现在我们考虑另一个在数据结构和类设计中常见的问题, 如何处理复制. 比如在实际工作中, 下面的代码是经常会用到的: 
```
ArrayList A(3), B;
ArrayList.insert(3, 0);
ArrayList.insert(2, 0);
ArrayList.insett(1, 0);

B = A;
```
这里我们先抛开一切, 最直观的要求就是希望完成这个赋值运算后, `B` 的内容要和 `A` 完全一样. 和缺省构造函数一样, 编译器默认也会提供一个缺省的赋值函数和一个被称为复制构造函数的东西, 后者专门完成下面这样的操作:
```
ArraryList C(A);
```
要求也是 `C` 的内容和 `A` 一样. 显然, 缺省提供复制和赋值, 在这个例子都不能完成我们的要求. 因为它们会把成员属性都对应复制一遍, 而在我们这里, `data` 是一个指针, 这样做的后果就是两个实例指向同一段内存数据. 这是万恶之源! 在类似动态内存分配的情况下, 我们需要手工设计这两个要求几乎一样的功能. 先扩展我们的类设计:

In [ ]:
#include <iostream>

class ArrayList
{
private:
    // C++ 11 以后允许给类成员赋初指, 且发生在任何构造函数之前.
    char* data = nullptr;
    // 用以表示数组大小, 必须靠程序员可靠维护.
    int size = 0;
public:
    // 这个 const 表示该成员函数不会修改类的任何数据.
    void printList() const;
    void makeEmpty();
    // 从实用角度, find 返回一个指针会更有意义, 所以这里也提供了动静两套版本.
    const char* find(char val) const ;  
    char* find(char val) ;
    // 原本的 find 保留, 但改名为 findIdx.
    int findIdx(char val) const;
    // 动态函数就不可以加 const 限制.
    void insert(char val, int pos);
    void remove(char val);
    // 不允许修改返回的指针, 也不允许函数修改类成员数据. 
    const char* findKth(int pos) const;
    // 这是上一个函数的重载, 允许修改返回的指针, 也允许修改函数成员的数据.
    char* findKth(int pos);
    ArrayList() { }
    ArrayList(int N) { data = new char[N]; };
    ~ArrayList() { if (data != nullptr) delete [] data; }
    // 复制构造函数, 这里对参数做了引用.
    ArrayList(const ArrayList &rhs);
    // 赋值运算的重载.
    ArrayList &operator=(const ArrayList &rhs);
};